# Tensorflow MLP
Here we will attempt to design a regression model for the data. First we will get the data from the new file then select which to use. The feature vector will be {b1, a2, frac, RatioTotalArea} and the target will be {eta c}. 

There are currently 4 options:
- entire unscaled dataset: 57841
- entire scaled dataset (from unscaled): 57841
- limited dataset: 42016
- limited and scaled dataset: 42016

I will use the limited and scaled dataset to start because I can randomise easily. The data is scaled using the MaxAbsScaler to start.

### I do think that we may be loosing data on the 2nd ellipse, so another feature may be necessary. (The info for the other dimension is there, but in the 'RatioTotalArea').

In [2]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from sklearn.preprocessing import MaxAbsScaler
import seaborn as sns
sns.set()

In [3]:
name = "LIM_scaled.csv"
dataset = pd.read_csv(name)
# data2.describe().transpose()
dataset.pop("Unnamed: 0")
dataset.describe().transpose()

,count,mean,std,min,25%,50%,75%,max
b1,42016.0,0.348088,0.328365,0.006000,0.060000,0.200000,0.666667,1.0
a2,42016.0,0.421380,0.332279,0.010000,0.100000,0.400000,0.700000,1.0
RatioTotalArea,42016.0,0.178203,0.185276,0.000197,0.034663,0.106253,0.275565,1.0
frac,42016.0,0.521135,0.318137,0.052632,0.210526,0.526316,0.842105,1.0
eta c,42016.0,0.228514,0.178317,0.004062,0.073706,0.186736,0.365193,1.0
